<a href="https://colab.research.google.com/github/bcury/Python-Projects/blob/main/Quantum_Information_Algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install Qiskit

     |████████████████████████████████| 6.1 MB 1.8 MB/s 
     |████████████████████████████████| 17.9 MB 538 kB/s 
     |████████████████████████████████| 237 kB 62.8 MB/s 
     |████████████████████████████████| 207 kB 43.9 MB/s 
     |████████████████████████████████| 2.1 MB 43.6 MB/s 
     |████████████████████████████████| 634 kB 42.9 MB/s 
     |████████████████████████████████| 1.4 MB 33.9 MB/s 
     |████████████████████████████████| 52 kB 1.2 MB/s 
     |████████████████████████████████| 49 kB 5.2 MB/s 
     |████████████████████████████████| 38.2 MB 1.1 MB/s 
     |████████████████████████████████| 943 kB 29.4 MB/s 
     |████████████████████████████████| 3.6 MB 45.9 MB/s 
     |████████████████████████████████| 6.3 MB 37.6 MB/s 
  Created wheel for Qiskit: filename=qiskit-0.32.1-py3-none-any.whl size=11776 sha256=49ab4b02ed938c0f59a49139e8b110d28bc6afe67d04aaa21e993e2af670eff6
  Stored in directory: /root/.cache/pip/wheels/0f/62/0a/c53eda1ead41c137c47c9730bc2771a8367b1ce00fb6

In [10]:
try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq --pre
    print("installed cirq.")



installing cirq...
     |████████████████████████████████| 504 kB 11.4 MB/s 
     |████████████████████████████████| 56 kB 3.8 MB/s 
     |████████████████████████████████| 1.6 MB 39.2 MB/s 
     |████████████████████████████████| 328 kB 47.6 MB/s 
     |████████████████████████████████| 47 kB 3.9 MB/s 
     |████████████████████████████████| 220 kB 66.5 MB/s 
     |████████████████████████████████| 52 kB 1.2 MB/s 
     |████████████████████████████████| 10.1 MB 64.7 MB/s 
     |████████████████████████████████| 145 kB 52.1 MB/s 
     |████████████████████████████████| 65 kB 3.1 MB/s 
     |████████████████████████████████| 49 kB 5.1 MB/s 
     |████████████████████████████████| 53 kB 1.6 MB/s 
     |████████████████████████████████| 97 kB 5.2 MB/s 
     |████████████████████████████████| 229 kB 53.0 MB/s 
     |████████████████████████████████| 15.7 MB 51.9 MB/s 
     |████████████████████████████████| 38.1 MB 269 kB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |███

In [15]:
import cirq_google as cg

# The Google Cloud Project id to use.
project_id = "" #@param {type:"string"}
processor_id = "" #@param {type:"string"}

from cirq_google.engine.qcs_notebook import get_qcs_objects_for_notebook
device_sampler = get_qcs_objects_for_notebook(project_id, processor_id)



Using a noisy simulator.


In [16]:
import cirq
import cirq_google
from google.auth.exceptions import DefaultCredentialsError
from google.api_core.exceptions import PermissionDenied

In [17]:
# Define a qubit at an arbitrary grid location.
qubit = cirq.GridQubit(0, 0)

In [19]:
 #Create a circuit (qubits start in the |0> state).
circuit = cirq.Circuit(
    cirq.X(qubit),                     # NOT gate.
    cirq.measure(qubit, key='result')  # Measurement.
)

print("Circuit:")
print(circuit)

Circuit:
(0, 0): ───X───M('result')───


Let's quickly use Cirq to simulate the circuit above

In [20]:
# Simulate the circuit, repeating 1000 times.
print("Simulating circuit using Cirq...\n")
results = cirq.sample(circuit, repetitions=1000)
print("Measurement results:")
print(results)

Simulating circuit using Cirq...

Measurement results:
result=11111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111

In [21]:
# Create an Engine client.
engine = cg.get_engine()

In [ ]:
processor = engine.get_processor('pacific')

# Print the device showing qubit connectivity for the Sycamore gateset.
device = processor.get_device([cg.SYC_GATESET])
print(device)

In [ ]:
valid_qubit = device.qubits[0]

# Re-map qubits and gates for Sycamore hardware.
syc_circuit = cg.optimized_for_sycamore(
    circuit,
    new_device=device,
    qubit_map=lambda q: valid_qubit) # Constant fn since there's only one qubit.

print(syc_circuit)

In [ ]:
# Upload the program and submit jobs to run in one call.
job = engine.run_sweep(
    program=syc_circuit,
    repetitions=10000,
    processor_ids=[processor.processor_id],
    gate_set=cg.SYC_GATESET)

print("Scheduled. View the job at: https://console.cloud.google.com/quantum/"
      "programs/{}?&project={}".format(job.program_id, project_id))

# Print out the results. This blocks until the results are returned.
results = [str(int(b)) for b in job.results()[0].measurements['result'][:, 0]]
print("Measurement results:\n")
print(''.join(results))

In [26]:
import cirq

def example():
    """Hello qubit example run against a quantum processor."""

    # Define a qubit.
    qubit = cirq.GridQubit(5, 2)

    # Create a circuit (qubits start in the |0> state).
    circuit = cirq.Circuit(
        cirq.X(qubit)**0.5,                # Square root of NOT.
        cirq.measure(qubit, key='result')  # Measurement.
    )

    # Create an Engine object.  This uses the project id of your
    # Google cloud project.
    project_id = 'your-project-id'